In [1]:
import pandas as pd
from src.topic_modelling.topic_model import TopicModel

/home/rainergo/.local/share/virtualenvs/RELATION-EXTRACTION-Yry67amm/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
df = pd.read_parquet('../data/df_ner_coref.parquet')
size_df = len(df.index)
print(size_df)

100


#### CONVERT CELL FORMAT

In [3]:
df = df.explode('new').reset_index(drop=False)

In [4]:
import numpy as np
df['top_sent'] = df['new'].str['sentence'].astype(object).replace(np.nan, pd.NA)

In [5]:
from functools import reduce

def mask_sent(comp_dict, mask: str = "Comp@Name@Placeholder") -> str:
    sent = pd.NA
    if isinstance(comp_dict, dict):
        replacements: dict = {ent['ent_text']: mask for ent in comp_dict['entities']}
        sent: str = comp_dict['sentence']
        sent = reduce(lambda x, kv: x.replace(*kv), replacements.items(), sent)
    return sent

In [6]:
df['top_sent_masked'] = df['new'].apply(mask_sent)

In [7]:
# df.to_parquet('df.parquet')
df.head(5)

,index,art_source,art_url,art_type,art_datetime,art_language,art_isin,art_company_name,art_title,art_text,art_author,art_abstract,art_video_url,pp_art_text,new,top_sent,top_sent_masked
0,646,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-05 15:01:00+02:00,DE,CH0013841017,Lonza Group AG,EQS-News: Lonza gibt Ergebnisse der Generalver...,Lonza gibt Ergebnisse der Generalversammlung 2...,<NA>,<NA>,<NA>,Lonza gibt Ergebnisse der Generalversammlung 2...,NaN,<NA>,<NA>
1,225,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-03 19:31:00+02:00,DE,DE000A14KEB5,home24 SE,EQS-Stimmrechte: home24 SE (deutsch),home24 SE: Veröffentlichung gemäß § 40 Abs. 1 ...,<NA>,<NA>,<NA>,EQS Stimmrechtsmitteilung: home24 SE. Stimmre...,NaN,<NA>,<NA>
2,3232,eqs-news,https://www.eqs-news.com/de/news/corporate/hyp...,Corporate,2023-05-08 07:05:00+02:00,DE,DE0005493365,Hypoport SE,Hypoport SE Hypoport SE: Hypoport startet mit ...,Konzernergebnis Q1 /23 Hypoport startet mit le...,<NA>,<NA>,<NA>,Konzernergebnis Q1 23. Hypoport startet mit l...,"{'entities': [{'comp_name': 'Hypoport SE', 'co...",Konzernergebnis Q1 23. Hypoport startet mit l...,Konzernergebnis Q1 23. Comp@Name@Placeholder ...
3,3232,eqs-news,https://www.eqs-news.com/de/news/corporate/hyp...,Corporate,2023-05-08 07:05:00+02:00,DE,DE0005493365,Hypoport SE,Hypoport SE Hypoport SE: Hypoport startet mit ...,Konzernergebnis Q1 /23 Hypoport startet mit le...,<NA>,<NA>,<NA>,Konzernergebnis Q1 23. Hypoport startet mit l...,"{'entities': [{'comp_name': 'Hypoport SE', 'co...","Ronald Slabke, Vorstandsvorsitzender der Hypop...","Ronald Slabke, Vorstandsvorsitzender der Comp@..."
4,3232,eqs-news,https://www.eqs-news.com/de/news/corporate/hyp...,Corporate,2023-05-08 07:05:00+02:00,DE,DE0005493365,Hypoport SE,Hypoport SE Hypoport SE: Hypoport startet mit ...,Konzernergebnis Q1 /23 Hypoport startet mit le...,<NA>,<NA>,<NA>,Konzernergebnis Q1 23. Hypoport startet mit l...,"{'entities': [{'comp_name': 'Hypoport SE', 'co...",Unter den gegebenen Rahmenbedingungen ist Hypo...,Unter den gegebenen Rahmenbedingungen ist Comp...


#### START TOPIC MODELING

In [8]:
top = TopicModel(df=df)

In [9]:
top.run()

GPU is used: True
Vectorizer.vectorize() was run.
DimReducer.reduce() was run!
Clusterer.cluster() was run!
DimReducer.reduce() was run!
Vectorizer.vectorize() was run.
Dimension of all_cluster_vectors: (10, 948)


In [11]:
top.df.cluster_label.value_counts()

cluster_label
0    87
6    23
3    22
5    20
9    19
1     9
7     8
4     8
8     7
2     6
Name: count, dtype: int64

In [12]:
top.df_data.head()

,top_sent,top_prep_sent,top_prep_sent_words,cluster_label,top_red_viz_vector,x,y,z
2,Konzernergebnis Q1 23. Hypoport startet mit l...,[konzernergebnis starten leicht wachstum koste...,"[konzernergebnis, starten, leicht, wachstum, k...",9,"[[-26.93161248572412, -10.30119435350977, 0.35...",-26.93,-10.30,0.35
3,"Ronald Slabke, Vorstandsvorsitzender der Hypop...",[vorstandsvorsitzender hypoport],"[vorstandsvorsitzender, hypoport]",0,"[[-11.087532021278799, 6.370232434643442, -9.1...",-11.09,6.37,-9.10
4,Unter den gegebenen Rahmenbedingungen ist Hypo...,[gegeben rahmenbedingung erfolgreich starten],"[gegeben, rahmenbedingung, erfolgreich, starten]",9,"[[-20.927279846468462, -13.770233417984265, 13...",-20.93,-13.77,13.08
6,"Zahlungsmittel, Zahlungsmittelaequivalente und...",[zahlungsmittel zahlungsmittelaequivalente boe...,"[zahlungsmittel, zahlungsmittelaequivalente, b...",5,"[[6.846739819435593, 12.169515286006693, -43.5...",6.85,12.17,-43.54
7,"Merus N.V., ein Unternehmen aus dem Bereich de...",[v unternehmen bereich klinisch onkologie inno...,"[v, unternehmen, bereich, klinisch, onkologie,...",2,"[[106.10661860969917, -6.896218063576454, 13.1...",106.11,-6.90,13.17


In [13]:
ind = 2
col = 'top_prep_sent'
df_len = len(top.df_data[col].loc[ind])
df_type = type(top.df_data[col].loc[ind])
df_element_type = type(top.df_data[col].loc[ind][0])
print(df_type, df_len, df_element_type)

<class 'list'> 1 <class 'str'>


In [14]:
top.df.head()

,index,art_source,art_url,art_type,art_datetime,art_language,art_isin,art_company_name,art_title,art_text,art_author,art_abstract,art_video_url,pp_art_text,new,top_sent,top_sent_masked,top_prep_sent_words,top_prep_sent,top_vector,top_red_vector,cluster_label,top_red_viz_vector
0,646,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-05 15:01:00+02:00,DE,CH0013841017,Lonza Group AG,EQS-News: Lonza gibt Ergebnisse der Generalver...,Lonza gibt Ergebnisse der Generalversammlung 2...,<NA>,<NA>,<NA>,Lonza gibt Ergebnisse der Generalversammlung 2...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,225,dpa-afx-compact,https://mobile.traderfox.com/news/dpa-compact/...,unt,2023-05-03 19:31:00+02:00,DE,DE000A14KEB5,home24 SE,EQS-Stimmrechte: home24 SE (deutsch),home24 SE: Veröffentlichung gemäß § 40 Abs. 1 ...,<NA>,<NA>,<NA>,EQS Stimmrechtsmitteilung: home24 SE. Stimmre...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,3232,eqs-news,https://www.eqs-news.com/de/news/corporate/hyp...,Corporate,2023-05-08 07:05:00+02:00,DE,DE0005493365,Hypoport SE,Hypoport SE Hypoport SE: Hypoport startet mit ...,Konzernergebnis Q1 /23 Hypoport startet mit le...,<NA>,<NA>,<NA>,Konzernergebnis Q1 23. Hypoport startet mit l...,"{'entities': [{'comp_name': 'Hypoport SE', 'co...",Konzernergebnis Q1 23. Hypoport startet mit l...,Konzernergebnis Q1 23. Comp@Name@Placeholder ...,"[konzernergebnis, starten, leicht, wachstum, k...",[konzernergebnis starten leicht wachstum koste...,"[[-2.727808793819026, -3.96094098297696, -3.40...","[[-1.3835460213872446, -0.6470582549067596, -0...",9,"[[-26.93161248572412, -10.30119435350977, 0.35..."
3,3232,eqs-news,https://www.eqs-news.com/de/news/corporate/hyp...,Corporate,2023-05-08 07:05:00+02:00,DE,DE0005493365,Hypoport SE,Hypoport SE Hypoport SE: Hypoport startet mit ...,Konzernergebnis Q1 /23 Hypoport startet mit le...,<NA>,<NA>,<NA>,Konzernergebnis Q1 23. Hypoport startet mit l...,"{'entities': [{'comp_name': 'Hypoport SE', 'co...","Ronald Slabke, Vorstandsvorsitzender der Hypop...","Ronald Slabke, Vorstandsvorsitzender der Comp@...","[vorstandsvorsitzender, hypoport]",[vorstandsvorsitzender hypoport],"[[-2.727808793819026, -3.96094098297696, -3.40...","[[-0.6341794088027193, 0.2477554519680205, -0....",0,"[[-11.087532021278799, 6.370232434643442, -9.1..."
4,3232,eqs-news,https://www.eqs-news.com/de/news/corporate/hyp...,Corporate,2023-05-08 07:05:00+02:00,DE,DE0005493365,Hypoport SE,Hypoport SE Hypoport SE: Hypoport startet mit ...,Konzernergebnis Q1 /23 Hypoport startet mit le...,<NA>,<NA>,<NA>,Konzernergebnis Q1 23. Hypoport startet mit l...,"{'entities': [{'comp_name': 'Hypoport SE', 'co...",Unter den gegebenen Rahmenbedingungen ist Hypo...,Unter den gegebenen Rahmenbedingungen ist Comp...,"[gegeben, rahmenbedingung, erfolgreich, starten]",[gegeben rahmenbedingung erfolgreich starten],"[[-2.727808793819026, -3.96094098297696, -3.40...","[[-1.177121653716793, -0.9093325205759918, 0.5...",9,"[[-20.927279846468462, -13.770233417984265, 13..."


In [15]:
df_len = len(top.df[col].loc[ind])
df_type = type(top.df[col].loc[ind])
df_element_type = type(top.df[col].loc[ind][0])
print(df_type, df_len, df_element_type)

<class 'list'> 1 <class 'str'>


In [16]:
top.df_cluster.head()

,x,y,z,top_sent,top_prep_sent,top_prep_sent_words,top_cluster_vector,most_frequent_words
cluster_label,,,,,,,,
0,-13.66,6.08,-19.00,"Ronald Slabke, Vorstandsvorsitzender der Hypop...",[vorstandsvorsitzender hypoport amerikanisch r...,"[vorstandsvorsitzender, hypoport, amerikanisch...","[[0.0, 0.0, 0.03562352499395473, 0.03562352499...","[(deutsch, 0.17811762496977365), (portfolio, 0..."
1,79.12,14.94,10.97,Unter dem Strich will ProSiebenSat.1 bereinigt...,[strich bereinigen sondereffekt abschreibung w...,"[strich, bereinigen, sondereffekt, abschreibun...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.04662524041201566...","[(euro, 0.6061281253562036), (million, 0.51287..."
2,74.84,0.22,-0.59,"Merus N.V., ein Unternehmen aus dem Bereich de...",[v unternehmen bereich klinisch onkologie inno...,"[v, unternehmen, bereich, klinisch, onkologie,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[(unternehmen, 0.4882730752626956), (klinisch,..."
3,-44.81,-68.25,90.97,Die RHOEN-KLINIKUM AG ist ein eigenstaendiges ...,[eigenstaendig unternehmen dach asklepios klin...,"[eigenstaendig, unternehmen, dach, asklepios, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[(co, 0.7654779007031424), (kunde, 0.173972250..."
4,7.13,9.40,-25.81,EQS Stimmrechtsmitteilung: Rheinmetall AG. St...,[stimmrechtsmitteilung stimmrechtsmitteilung s...,"[stimmrechtsmitteilung, stimmrechtsmitteilung,...","[[0.0, 0.10314212462587934, 0.0, 0.0, 0.0, 0.0...","[(instone, 0.618852747755276), (stimmrechtsmit..."


In [17]:
df_cluster_len = len(top.df_cluster[col].loc[ind])
df_cluster_type = type(top.df_cluster[col].loc[ind])
df_cluster_element_type = type(top.df_cluster[col].loc[ind][0])
print(df_cluster_type, df_cluster_len, df_cluster_element_type)

<class 'list'> 1 <class 'str'>


In [18]:
df_cluster_type == df_type

True

In [19]:
# top.df_data.head()

In [20]:
from copy import deepcopy
df_data = deepcopy(top.df)

In [21]:
df_data[["x", "y", "z"]] = df_data.apply(lambda x: np.squeeze(x.top_red_viz_vector).tolist() if x.top_red_viz_vector is not pd.NA else [pd.NA, pd.NA, pd.NA], axis=1, result_type='expand')

In [22]:
# df_data.head()

In [23]:
df_data = df_data.dropna(subset=['new'])

In [24]:
df_test = df_data.groupby('cluster_label').agg({'x':'mean', 'y':'mean', 'z':'mean', 'top_sent': ' '.join, 'top_prep_sent': lambda x: [' '.join(x.apply(' '.join))], 'top_prep_sent_words': lambda val: [x for xs in val for x in xs]})

df_test



,x,y,z,top_sent,top_prep_sent,top_prep_sent_words
cluster_label,,,,,,
0,-13.66,6.08,-19.00,"Ronald Slabke, Vorstandsvorsitzender der Hypop...",[vorstandsvorsitzender hypoport amerikanisch r...,"[vorstandsvorsitzender, hypoport, amerikanisch..."
1,79.12,14.94,10.97,Unter dem Strich will ProSiebenSat.1 bereinigt...,[strich bereinigen sondereffekt abschreibung w...,"[strich, bereinigen, sondereffekt, abschreibun..."
2,74.84,0.22,-0.59,"Merus N.V., ein Unternehmen aus dem Bereich de...",[v unternehmen bereich klinisch onkologie inno...,"[v, unternehmen, bereich, klinisch, onkologie,..."
3,-44.81,-68.25,90.97,Die RHOEN-KLINIKUM AG ist ein eigenstaendiges ...,[eigenstaendig unternehmen dach asklepios klin...,"[eigenstaendig, unternehmen, dach, asklepios, ..."
4,7.13,9.40,-25.81,EQS Stimmrechtsmitteilung: Rheinmetall AG. St...,[stimmrechtsmitteilung stimmrechtsmitteilung s...,"[stimmrechtsmitteilung, stimmrechtsmitteilung,..."
5,-16.04,-2.63,-59.72,"Zahlungsmittel, Zahlungsmittelaequivalente und...",[zahlungsmittel zahlungsmittelaequivalente boe...,"[zahlungsmittel, zahlungsmittelaequivalente, b..."
6,67.56,5.26,33.74,Die Jahreshauptversammlung des Unternehmens is...,[jahreshauptversammlung unternehmen mai planen...,"[jahreshauptversammlung, unternehmen, mai, pla..."
7,-36.32,99.37,49.85,Beta Systems Software AG: Vorstandsmitglied Dr...,[software vorstandsmitglied amt niederlegen so...,"[software, vorstandsmitglied, amt, niederlegen..."
8,28.96,5.07,-2.47,"Damit konnte das Unternehmen, trotz eines einm...",[unternehmen einmalig sondereffekt hoehe mio e...,"[unternehmen, einmalig, sondereffekt, hoehe, m..."


In [25]:
# print(len(df_test.top_prep_sent_words.loc[0]))
# print(type(df_test.top_prep_sent_words.loc[1]))

In [26]:
# top.df_data

In [27]:
# top.df_cluster

In [28]:
# top.df_cluster_doc_term_matrix